# Building Predictive Model

In [2]:
import pandas as pd
import numpy as np
import os

## Import Data

In [3]:
processed_data_path = os.path.join(os.path.pardir, "data", "processed")
train_file_path = os.path.join(processed_data_path, "train.csv")
test_file_path = os.path.join(processed_data_path, "test.csv")

In [4]:
train_df = pd.read_csv(train_file_path, index_col="PassengerId")
test_df = pd.read_csv(test_file_path, index_col="PassengerId")

In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 33 columns):
Age                   891 non-null float64
Fare                  891 non-null float64
FamilySize            891 non-null int64
IsMother              891 non-null int64
IsMale                891 non-null int64
Deck_A                891 non-null float64
Deck_B                891 non-null float64
Deck_C                891 non-null float64
Deck_D                891 non-null float64
Deck_E                891 non-null float64
Deck_F                891 non-null float64
Deck_G                891 non-null float64
Deck_Z                891 non-null float64
Pclass_1              891 non-null float64
Pclass_2              891 non-null float64
Pclass_3              891 non-null float64
Title_Lady            891 non-null float64
Title_Master          891 non-null float64
Title_Miss            891 non-null float64
Title_Mr              891 non-null float64
Title_Mrs             891 non-null float6

In [6]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 892 to 1309
Data columns (total 32 columns):
Age                   418 non-null float64
Fare                  418 non-null float64
FamilySize            418 non-null int64
IsMother              418 non-null int64
IsMale                418 non-null int64
Deck_A                418 non-null float64
Deck_B                418 non-null float64
Deck_C                418 non-null float64
Deck_D                418 non-null float64
Deck_E                418 non-null float64
Deck_F                418 non-null float64
Deck_G                418 non-null float64
Deck_Z                418 non-null float64
Pclass_1              418 non-null float64
Pclass_2              418 non-null float64
Pclass_3              418 non-null float64
Title_Lady            418 non-null float64
Title_Master          418 non-null float64
Title_Miss            418 non-null float64
Title_Mr              418 non-null float64
Title_Mrs             418 non-null flo

## Data Preparation

In [7]:
X = train_df.loc[:,:"AgeState_Child"].as_matrix().astype("float")
Y = train_df["Survived"].ravel()  # Creates flattened 1D array

In [8]:
print(X.shape, Y.shape)

(891, 32) (891,)


In [9]:
from sklearn.model_selection import train_test_split  # sklearn.model_selection from sklearn 0.19+
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(712, 32) (179, 32)
(712,) (179,)


In [10]:
import sklearn

In [11]:
sklearn.__version__

'0.19.1'

## Building the baseline model

In [12]:
from sklearn.dummy import DummyClassifier

In [13]:
dummy_model = DummyClassifier(strategy="most_frequent", random_state=0)

In [14]:
dummy_model.fit(X_train, y_train)

DummyClassifier(constant=None, random_state=0, strategy='most_frequent')

In [15]:
print("Score for baseline model = {0:.2f}".format(dummy_model.score(X_test, y_test)))

Score for baseline model = 0.61


In [16]:
# Performance metrics
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score

In [17]:
print("Accuracy for baseline model = {0:.2f}".format(accuracy_score(y_test, dummy_model.predict(X_test))))

Accuracy for baseline model = 0.61


## First Kaggle submission

In [18]:
test_X = test_df.as_matrix().astype("float")

In [19]:
predictions = dummy_model.predict(test_X)

In [21]:
df_submission = pd.DataFrame({"PassengerId": test_df.index, "Survived": predictions})

In [23]:
df_submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [24]:
submission_data_path = os.path.join(os.path.pardir, "data", "external")
submission_file_path = os.path.join(submission_data_path, "01_dummy.csv")

In [25]:
df_submission.to_csv(submission_file_path, index=False)

In [26]:
def get_submission_file(model, filename):
    test_X = test_df.as_matrix().astype("float")
    predictions = model.predict(test_X)
    
    df_submission = pd.DataFrame({"PassengerId": test_df.index, "Survived": predictions})
    
    submission_data_path = os.path.join(os.path.pardir, "data", "external")
    submission_file_path = os.path.join(submission_data_path, filename)
    df_submission.to_csv(submission_file_path, index=False)

## Logistic regression model

In [27]:
from sklearn.linear_model import LogisticRegression

In [28]:
model_lr_1 = LogisticRegression(random_state=0)

In [30]:
model_lr_1.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [31]:
print("Score for LR model: {0:.2f}".format(model_lr_1.score(X_test, y_test)))

Score for LR model: 0.83


In [33]:
predictions = model_lr_1.predict(X_test)
print("Accuracy score: {0:.2f}".format(accuracy_score(y_test, predictions)))
print("Confusion matrix:\n{0}".format(confusion_matrix(y_test, predictions)))
print("Precision score: {0:.2f}".format(precision_score(y_test, predictions)))
print("Recall score: {0:.2f}".format(recall_score(y_test, predictions)))

Accuracy score: 0.83
Confusion matrix:
[[95 15]
 [15 54]]
Precision score: 0.78
Recall score: 0.78


In [34]:
model_lr_1.coef_

array([[-0.02840159,  0.00455671, -0.50030169,  0.61990192, -0.81410923,
         0.12811061, -0.17277133, -0.39401076,  0.52243768,  1.09976697,
         0.40349376, -0.18343938, -0.30036674,  0.96580249,  0.48260674,
        -0.34518843,  0.28231021,  1.21937961,  0.56397096, -1.446179  ,
         1.07104886, -0.11413839, -0.47317144,  0.16296976,  0.24742886,
         0.280103  ,  0.41271918,  0.49208175,  0.46185058,  0.14928848,
         0.37244895,  0.73077185]])

## Second Kaggle Submission

In [35]:
get_submission_file(model_lr_1, "02_lr.csv")

## Hyperparameter Optimization

In [36]:
model_lr = LogisticRegression(random_state=0)

In [37]:
from sklearn.model_selection import GridSearchCV

In [38]:
parameters = {"C": [1.0, 10.0, 50.0, 100.0, 1000.0], "penalty": ["l1", "l2"]}
clf = GridSearchCV(model_lr, param_grid=parameters, cv=3)

In [40]:
clf.fit(X_train, y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'penalty': ['l1', 'l2'], 'C': [1.0, 10.0, 50.0, 100.0, 1000.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [42]:
clf.best_params_

{'C': 1.0, 'penalty': 'l1'}

In [43]:
clf.best_score_

0.8328651685393258

In [44]:
print("Score for logistic regression model: {0:.2f}".format(clf.score(X_test, y_test)))

Score for logistic regression model: 0.83


## Third Kaggle Submission

In [45]:
get_submission_file(clf, "03_lr.csv")

## Feature Normalization and Standardization

In [46]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

### Feature Normalization

In [47]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [48]:
X_train_scaled[:,0].min(), X_train_scaled[:,0].max()

(0.0, 1.0)

In [49]:
# Normalize test data
X_test_scaled = scaler.fit_transform(X_test)

### Feature Standardization

In [51]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Create Model after Standardization

In [53]:
model_lr = LogisticRegression()
parameters = {"C": [0.5, 1.0, 2.0, 5.0, 10.0, 50.0, 100.0], "penalty": ["l1", "l2"]}
clf = GridSearchCV(model_lr, param_grid=parameters, cv=3)
clf.fit(X_train_scaled, y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'penalty': ['l1', 'l2'], 'C': [0.5, 1.0, 2.0, 5.0, 10.0, 50.0, 100.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [54]:
clf.best_params_

{'C': 0.5, 'penalty': 'l1'}

In [55]:
clf.best_score_

0.8230337078651685

In [56]:
print("Score for standardized model: {0:.2f}".format(clf.score(X_test_scaled, y_test)))

Score for standardized model: 0.84
